In [27]:
import numpy as np
import pandas as pd
import sklearn 
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv("C:/Users/Sande\Desktop/bankmarketing.csv")
df.sample(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
28051,31,blue-collar,single,basic.9y,no,no,no,telephone,apr,thu,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.483,5099.1,no
39163,36,management,single,university.degree,no,no,no,telephone,mar,wed,...,1,999,0,nonexistent,-1.8,93.369,-34.8,0.655,5008.7,yes
86,41,management,married,basic.6y,no,no,no,telephone,may,mon,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
17423,47,housemaid,married,basic.4y,unknown,no,no,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
116,52,admin.,divorced,university.degree,no,no,no,telephone,may,mon,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [4]:
df.shape

(41188, 21)

In [5]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [6]:
df.duplicated().sum()

np.int64(12)

In [7]:
df.drop_duplicates(inplace  = True)

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df.sample(1)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
6840,35,blue-collar,married,basic.6y,no,no,no,telephone,may,wed,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [10]:
print(df['job'].unique())
print(df['marital'].unique())
print(df['education'].unique())
print(df['default'].unique())
print(df['housing'].unique())
print(df['loan'].unique())
print(df['contact'].unique())
print(df['month'].unique())
print(df['day_of_week'].unique())
print(df['poutcome'].unique())
print(df['y'].unique())

['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
['married' 'single' 'divorced' 'unknown']
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
['no' 'unknown' 'yes']
['no' 'yes' 'unknown']
['no' 'yes' 'unknown']
['telephone' 'cellular']
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
['mon' 'tue' 'wed' 'thu' 'fri']
['nonexistent' 'failure' 'success']
['no' 'yes']


In [11]:
df.loc[df['job'] == "housemaid",['job']] = 1
df.loc[df['job'] == 'services',['job']] = 2
df.loc[df['job'] == 'admin.',['job']] = 3
df.loc[df['job'] == 'blue-collar',['job']] = 3
df.loc[df['job'] == 'technician',['job']] = 4
df.loc[df['job'] == 'retired',['job']] = 2
df.loc[df['job'] == 'management',['job']] = 2
df.loc[df['job'] == 'unemployed',['job']] = 1
df.loc[df['job'] == 'self-employed',['job']] = 2
df.loc[df['job'] == 'unknown',['job']] = 1
df.loc[df['job'] == 'entrepreneur',['job']] = 4
df.loc[df['job'] == 'student',['job']] = 1


In [12]:
print(df['job'].value_counts())

job
3    19672
2    10030
4     8195
1     3279
Name: count, dtype: int64


In [13]:
df.loc[df['education'] == 'university.degree',['education']] =  4
df.loc[df['education'] == 'high.school',['education']] = 4 
df.loc[df['education'] == 'basic.9y',['education']] = 3
df.loc[df['education'] == 'professional.course',['education']] = 4
df.loc[df['education'] == 'basic.4y',['education']] = 3  
df.loc[df['education'] == 'basic.6y',['education']] = 3
df.loc[df['education'] == 'unknown',['education']] = 1 
df.loc[df['education'] == 'illiterate',['education']] = 2 

In [14]:
df['education'].value_counts()

education
4    26916
3    12512
1     1730
2       18
Name: count, dtype: int64

In [15]:
df['poutcome'].value_counts()

poutcome
nonexistent    35551
failure         4252
success         1373
Name: count, dtype: int64

In [16]:
df.loc[df['poutcome'] == 'success',['poutcome']] = 3 
df.loc[df['poutcome'] == 'nonexistent',['poutcome']] = 2 
df.loc[df['poutcome'] == 'failure',['poutcome']] = 1


In [17]:
df['poutcome'].value_counts()

poutcome
2    35551
1     4252
3     1373
Name: count, dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41176 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41176 non-null  int64  
 1   job             41176 non-null  object 
 2   marital         41176 non-null  object 
 3   education       41176 non-null  object 
 4   default         41176 non-null  object 
 5   housing         41176 non-null  object 
 6   loan            41176 non-null  object 
 7   contact         41176 non-null  object 
 8   month           41176 non-null  object 
 9   day_of_week     41176 non-null  object 
 10  duration        41176 non-null  int64  
 11  campaign        41176 non-null  int64  
 12  pdays           41176 non-null  int64  
 13  previous        41176 non-null  int64  
 14  poutcome        41176 non-null  object 
 15  emp.var.rate    41176 non-null  float64
 16  cons.price.idx  41176 non-null  float64
 17  cons.conf.idx   41176 non-null  floa

In [19]:
df.sample(1)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
1917,32,2,married,4,no,yes,no,telephone,may,fri,...,2,999,0,2,1.1,93.994,-36.4,4.855,5191.0,no


In [24]:
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

job
marital
education
default
housing
loan
contact
month
day_of_week
poutcome
y


In [25]:
for i in df.columns:
    if df[i].dtype=='int64':
        print(i)

age
duration
campaign
pdays
previous


In [26]:
cat_features = ['marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'y']
num_features = ['age', 'duration', 'campaign', 'pdays', 'previous']

In [28]:
cat_transformer = OneHotEncoder(handle_unknown = 'ignore')
num_transformer = StandardScaler()

In [30]:
prepro = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, cat_features),
        ('num', num_transformer, num_features)
    ]
)

In [31]:
pipeline = Pipeline(steps=[
    ('prepro',prepro ),
    ('clusterring', KMeans(n_clusters=4, random_state = 42)) 
])

In [32]:
pipeline.fit(df)

Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['marital', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'y']),
                                                 ('num', StandardScaler(),
                                                  ['age', 'duration',
                                                   'campaign', 'pdays',
                                                   'previous'])])),
                ('clusterring', KMeans(n_clusters=4, random_state=42))])

In [33]:
cluster_labels = pipeline.predict(df)
df['cluster'] = cluster_labels

In [36]:
df.head(10)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,cluster
0,56,1,married,3,no,no,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,2
1,57,2,married,4,unknown,no,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,2
2,37,2,married,4,no,yes,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,1
3,40,3,married,3,no,no,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,1
4,56,2,married,4,no,no,yes,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,2
5,45,2,married,3,unknown,no,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,2
6,59,3,married,4,no,no,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,2
7,41,3,married,1,unknown,no,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,2
8,24,4,single,4,no,yes,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,1
9,25,2,single,4,no,yes,no,telephone,may,mon,...,999,0,2,1.1,93.994,-36.4,4.857,5191.0,no,1
